In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas_usaddress
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the postgres connection information from os file. 

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 

In [2]:

real_acct_df = pd.read_csv('../real_estate_data/real_acct_owner/real_acct.txt', sep='\t',lineterminator='\r',header=0,encoding='latin1',low_memory=False)

In [3]:
real_acct_df.head(2)

,acct,yr,mailto,mail_addr_1,mail_addr_2,mail_city,mail_state,mail_zip,mail_country,undeliverable,...,protested,certified_date,rev_dt,rev_by,new_own_dt,lgl_1,lgl_2,lgl_3,lgl_4,jurs
0,\n0010010000013,2021.0,CITY OF HOUSTON,PO BOX 1562,,HOUSTON,TX,77251-1562,,N,...,N,08/06/2021,01/01/2016,01391,01/02/1988,ALL BLK 1,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 265 268 576
1,\n0010020000001,2021.0,CURRENT OWNER,1717 SAINT JAMES PLACE STE 112,,HOUSTON,TX,77056-3412,,N,...,Y,NaN,05/28/2020,00203,10/31/2012,TR 15 BLK 2,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 265 268 576


In [4]:
columns = real_acct_df.columns
columns

Index(['acct', 'yr', 'mailto', 'mail_addr_1', 'mail_addr_2', 'mail_city',
       'mail_state', 'mail_zip', 'mail_country', 'undeliverable', 'str_pfx',
       'str_num', 'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
       'site_addr_1', 'site_addr_2', 'site_addr_3', 'state_class',
       'school_dist', 'map_facet', 'key_map', 'Neighborhood_Code',
       'Neighborhood_Grp', 'Market_Area_1', 'Market_Area_1_Dscr',
       'Market_Area_2', 'Market_Area_2_Dscr', 'econ_area', 'econ_bld_class',
       'center_code', 'yr_impr', 'yr_annexed', 'splt_dt', 'dsc_cd', 'nxt_bld',
       'bld_ar', 'land_ar', 'acreage', 'Cap_acct', 'shared_cad', 'land_val',
       'bld_val', 'x_features_val', 'ag_val', 'assessed_val', 'tot_appr_val',
       'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
       'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
       'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
       'value_status', 'noticed', 'notice_dt', 'protested', 'certif

In [5]:
# new_cols = ['acct', 'str_pfx', 'str_num',
#        'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
#        'site_addr_1', 'site_addr_2', 'site_addr_3','bld_val', 'x_features_val', 'ag_val', 
#        'assessed_val', 'tot_appr_val',
#        'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
#        'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
#        'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
#        'value_status', 'certified_date']

In [6]:
new_cols = ['acct','str_num','str_num_sfx','str','str_sfx','str_sfx_dir','str_unit','site_addr_1', 'site_addr_2',
'site_addr_3','bld_ar','land_ar','tot_appr_val','tot_mkt_val']

In [7]:
# remove the columns that we don't need
real_acct_df = real_acct_df[new_cols]
real_acct_df.head()

,acct,str_num,str_num_sfx,str,str_sfx,str_sfx_dir,str_unit,site_addr_1,site_addr_2,site_addr_3,bld_ar,land_ar,tot_appr_val,tot_mkt_val
0,\n0010010000013,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,44431.0,0.0,0.0
1,\n0010020000001,907.0,,COMMERCE,ST,,,907 COMMERCE ST,HOUSTON,77002,0.0,5001.0,309120.0,309120.0
2,\n0010020000003,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,18121.0,889398.0,889398.0
3,\n0010020000004,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,9061.0,444699.0,444699.0
4,\n0010020000013,921.0,,COMMERCE,ST,,,921 COMMERCE ST,HOUSTON,77002,0.0,3001.0,0.0,0.0


In [8]:
# remove the \n from the account column
def clean(x):
    x = x.replace('\n','')
    return x

In [9]:
real_acct_df['acct'] = real_acct_df['acct'].apply(clean)

In [10]:

real_acct_df.dtypes

acct             object
str_num         float64
str_num_sfx      object
str              object
str_sfx          object
str_sfx_dir      object
str_unit         object
site_addr_1      object
site_addr_2      object
site_addr_3      object
bld_ar          float64
land_ar         float64
tot_appr_val    float64
tot_mkt_val     float64
dtype: object

In [11]:
real_acct_df.head(2)

,acct,str_num,str_num_sfx,str,str_sfx,str_sfx_dir,str_unit,site_addr_1,site_addr_2,site_addr_3,bld_ar,land_ar,tot_appr_val,tot_mkt_val
0,0010010000013,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,44431.0,0.0,0.0
1,0010020000001,907.0,,COMMERCE,ST,,,907 COMMERCE ST,HOUSTON,77002,0.0,5001.0,309120.0,309120.0


In [12]:
# load 5 million rows to local database
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/Houston_real_estate')


In [13]:
# load df to database
real_acct_df.to_sql('real_acct',engine,index=False,if_exists='replace')

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

## Start Stan's web scraping from HAR

In [ ]:
def fix_num(num):
    return float(re.sub('[^0-9\.]', "", num))

def fix_num_int(num):
    return int(re.sub('[^0-9\.]', "", num))

In [ ]:
#read Haris county zip codes
zip_code_data=pd.read_csv("harris_county_zip_codes.csv")
zip_code_list=list(zip_code_data['zip_code'].map(str))

In [ ]:
#set up chrome browser
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# #define empty dataframe to appand data to
# har_df = pd.DataFrame(columns = ['mls', 'address', 'bld_sft','lot_sft','price', 'zip'])
# #search patterns for building and land square footage
# bldgsft_pattern = re.compile(r'Building Sqft')
# lotsft_pattern = re.compile(r'Lot Sqft')

# #define a subroutine to extract data from house structure and append it to a dataframe
# def extract_from_house(house):
#     global har_df
#     asking_price=fix_num(house.find('div', class_="price").text)
#     try:
#         bldg_sft=fix_num(house.find('div', class_="mp_features").find(text=bldgsft_pattern).__dict__["previous_sibling"].text)
#     except AttributeError:
#         bldg_sft=0
#     try:
#         lot_sft=fix_num(house.find('div', class_="mp_features").find(text=lotsft_pattern).__dict__["previous_sibling"].text)
#     except AttributeError:
#         lot_sft=0
#     addr=house.find('a', class_="address").text
#     mls_no=fix_num_int(house.find('div', class_="mpi_mls").text)
#     property_zip=current_zip
#     har_df = har_df.append({'mls' : mls_no, 'address' : addr, 'bld_sft' : bldg_sft, 'lot_sft': lot_sft, 'price': asking_price, 'zip': current_zip }, ignore_index = True)

# # cycle through all Harris county zip codes
# for current_zip in zip_code_list:
#     time.sleep(10)
#     print(current_zip)
#     #define starting url for current zip code
#     url=f'https://www.har.com/zipcode_{current_zip}/realestate/for_sale'
#     browser.visit(url)
#     html = browser.html
#     soup = bs(html, 'html.parser')
#     #Scrape the houses for the current search page
#     houses=soup.find_all('div', class_="prop_item")
        
#     for house in houses:
#         extract_from_house(house)

#     #figure out how many pages we will have to click through
#     navigation=soup.find_all('div', class_="pagination")
#     number_of_pages=len(navigation[0].ul.find_all('li'))

#     #click through pages 2 through the last page 
#     for i in range(2, number_of_pages+1):
#         nextlink=browser.links.find_by_text(str(i))
#         time.sleep(5)
#         #figure out ambiguity to what is the next search page link
#         if len(nextlink)==1:
#             nextlink.click()
#         else:
#             nextlink[1].click()
        
#         #extract houses from the current page and append to a data frame
#         html = browser.html
#         soup = bs(html, 'html.parser')
#         houses=soup.find_all('div', class_="prop_item")
#         for house in houses:
#             extract_from_house(house)
            
# #save scraped results for later use            
# har_df.to_csv('har_data.csv',index=False)

In [ ]:
# current_zip='77401'
# bldgsft_pattern = re.compile(r'Building Sqft')
# lotsft_pattern = re.compile(r'Lot Sqft')
# for house in houses:
#     asking_price=fix_num(house.find('div', class_="price").text)
#     bldg_sft=fix_num(house.find('div', class_="mp_features").find(text=bldgsft_pattern).__dict__["previous_sibling"].text)
#     lot_sft=fix_num(house.find('div', class_="mp_features").find(text=lotsft_pattern).__dict__["previous_sibling"].text)
#     addr=house.find('a', class_="address").text
#     mls_no=fix_num_int(house.find('div', class_="mpi_mls").text)
#     property_zip=current_zip
#     har_df = har_df.append({'mls' : mls_no, 'address' : addr, 'bld_sft' : bldg_sft, 'lot_sft': lot_sft, 'price': asking_price, 'zip': current_zip }, ignore_index = True)
    

In [15]:
# Load cities to a dataframe
har = pd.read_csv('har_data.csv')
har_df = pd.DataFrame(har)
har_df.head()


,mls,address,bld_sft,lot_sft,price,zip
0,17777272,"1211 Caroline Street PH2002, Houston, TX 77002",2433.0,0.0,1745000.0,77002
1,54714244,"1211 Caroline Street #1903, Houston, TX 77002",2579.0,0.0,1490000.0,77002
2,8077568,"915 Franklin Street 9F, Houston, TX 77002",2380.0,0.0,1170000.0,77002
3,43392628,"2000 Bagby Street #15400, Houston, TX 77002",2359.0,0.0,1100000.0,77002
4,64069960,"1211 Caroline st #1105, Houston, TX 77002",1235.0,0.0,650000.0,77002


In [16]:
# Parse address into multiple columns to enable query with hcad data
address_df = pandas_usaddress.tag(har_df, ['address'])
address_df.head(2)

,mls,address,bld_sft,lot_sft,price,zip,AddressNumber,AddressNumberPrefix,AddressNumberSuffix,BuildingName,...,StreetNamePostDirectional,StreetNamePostModifier,StreetNamePostType,SubaddressIdentifier,SubaddressType,USPSBoxGroupID,USPSBoxGroupType,USPSBoxID,USPSBoxType,ZipCode
0,17777272,"1211 Caroline Street PH2002, Houston, TX 77002",2433.0,0.0,1745000.0,77002,1211,NaN,NaN,NaN,...,NaN,NaN,street,NaN,NaN,NaN,NaN,NaN,NaN,77002
1,54714244,"1211 Caroline Street #1903, Houston, TX 77002",2579.0,0.0,1490000.0,77002,1211,NaN,NaN,NaN,...,NaN,NaN,street,NaN,NaN,NaN,NaN,NaN,NaN,77002


In [17]:
#make column titles all lower case for ease of use in posgres
address_df.columns= address_df.columns.str.strip().str.lower()

In [18]:
address_df.head(2)

,mls,address,bld_sft,lot_sft,price,zip,addressnumber,addressnumberprefix,addressnumbersuffix,buildingname,...,streetnamepostdirectional,streetnamepostmodifier,streetnameposttype,subaddressidentifier,subaddresstype,uspsboxgroupid,uspsboxgrouptype,uspsboxid,uspsboxtype,zipcode
0,17777272,"1211 Caroline Street PH2002, Houston, TX 77002",2433.0,0.0,1745000.0,77002,1211,NaN,NaN,NaN,...,NaN,NaN,street,NaN,NaN,NaN,NaN,NaN,NaN,77002
1,54714244,"1211 Caroline Street #1903, Houston, TX 77002",2579.0,0.0,1490000.0,77002,1211,NaN,NaN,NaN,...,NaN,NaN,street,NaN,NaN,NaN,NaN,NaN,NaN,77002


In [21]:
# load to postgre
address_df.to_sql('har_address',engine,index=False,if_exists='replace')


In [22]:
# load to postgre
har_df.to_sql('har',engine,index=False,if_exists='replace')

In [23]:
# change column data type to text
engine.execute('''
alter table real_acct 
alter column str_num type varchar;
''')

In [24]:
# change column data type to text
engine.execute('''
alter table real_acct 
alter column str_unit type varchar;
''')

In [32]:
# query postgres and store in dataframe
har_vs_hcad_df = pd.read_sql('''
select ha.mls, r.str_num,r.str,r.site_addr_1,r.site_addr_3,ha.price as "har price", r.tot_appr_val as "hcad appraised value",
r.tot_mkt_val as "hcad total mkt value"
from har_address ha, real_acct r
where ha.zipcode = r.site_addr_3
and upper(ha.streetname) = upper(r.str)
and ha.addressnumber = r.str_num
''',engine)

In [36]:
# change data to None where str_unit is blank space or null
engine.execute('''
update real_acct 
set str_unit = 'None'
where length(str_unit) < 2;
''')
engine.execute('''
update real_acct 
set str_unit = 'None'
where str_unit is null;
''')



In [35]:
har_vs_hcad_df

,mls,str_num,str,site_addr_1,site_addr_3,har price,hcad appraised value,hcad total mkt value
0,58551451,1211,CAROLINE,1211 CAROLINE ST 1108,77002,400000.0,355000.0,355000.0
1,70136588,1211,CAROLINE,1211 CAROLINE ST 1108,77002,495000.0,355000.0,355000.0
2,64069960,1211,CAROLINE,1211 CAROLINE ST 1108,77002,650000.0,355000.0,355000.0
3,12045066,1211,CAROLINE,1211 CAROLINE ST 1108,77002,477500.0,355000.0,355000.0
4,54714244,1211,CAROLINE,1211 CAROLINE ST 1108,77002,1490000.0,355000.0,355000.0
...,...,...,...,...,...,...,...,...
147665,29197014,260,EL DORADO,260 EL DORADO BLVD 1113,77598,81999.0,67227.0,67227.0
147666,98981628,17734,FIFE,17734 FIFE LN,77598,195000.0,207243.0,207243.0
147667,67032686,2815,HERITAGE COLONY,2815 HERITAGE COLONY DR,77598,225000.0,162967.0,162967.0
147668,55844312,215,LINDA,215 LINDA LN,77598,336000.0,226842.0,226842.0


In [37]:
#This query  takes into account apartment numbers as well.
har_vs_hcad_df1 = pd.read_sql('''
select ha.mls, r.str_num,r.str,r.str_unit,r.site_addr_1,ha.address,r.site_addr_3,
ha.price as "har price", r.tot_appr_val as "hcad appraised value",
r.tot_mkt_val as "hcad total mkt value"
from har_address ha, real_acct r
where ha.zipcode = r.site_addr_3
and upper(ha.streetname) = upper(r.str)
and ha.addressnumber = r.str_num
and ha.occupancyidentifier=r.str_unit
''',engine)

In [38]:
har_vs_hcad_df1

,mls,str_num,str,str_unit,site_addr_1,address,site_addr_3,har price,hcad appraised value,hcad total mkt value
0,76911703,6633,AIRPORT,1001,6633 W AIRPORT BLVD 1001,"6633 W Airport Boulevard #1001, Houston, TX 77035",77035,69900.0,50512.0,50512.0
1,74667065,917,MAIN,1003,917 MAIN ST 1003,"917 Main Street #1003, Houston, TX 77002",77002,178000.0,166255.0,166255.0
2,20714853,2203,DORRINGTON,101,2203 DORRINGTON ST 101,"2203 Dorrington Street #101, Houston, TX 77030",77030,209000.0,174545.0,174545.0
3,29441069,7575,CAMBRIDGE,101,7575 CAMBRIDGE ST 101,"7575 Cambridge Street #101, Houston, TX 77054",77054,165000.0,136213.0,136213.0
4,25557317,2230,PINEY POINT,101,2230 S PINEY POINT RD 101,"2230 S Piney Point Road #101, Houston, TX 77063",77063,175000.0,137367.0,137367.0
...,...,...,...,...,...,...,...,...,...,...
714,78174542,2000,BAGBY,9415,2000 BAGBY ST 9415,"2000 Bagby Street #9415, Houston, TX 77002",77002,339900.0,323513.0,323513.0
715,16595788,2000,BAGBY,9416,2000 BAGBY ST 9416,"2000 Bagby Street #9416, Houston, TX 77002",77002,404000.0,430657.0,430657.0
716,78213391,2000,BAGBY,9424,2000 BAGBY ST 9424,"2000 Bagby Street #9424, Houston, TX 77002",77002,365000.0,401294.0,401294.0
717,49610448,9535,PAGEWOOD,9535,9535 PAGEWOOD LN 9535,"9535 Pagewood Lane #9535, Houston, TX 77063",77063,115000.0,99872.0,99872.0


In [30]:
# end session and close the browser
# browser.quit()